In [ ]:
from openai import OpenAI
from config import DEEPSEEK_API_KEY, OPENROUTER_API_KEY
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,#Your model API key here
)
messages=[ {"role":"system","content":
       """
You are a helpful assistant. Solve this puzzle for me.
There are three pegs and n disks of different sizes stacked on the first peg.
The disks are numbered from 1 (smallest) to n (largest). Disk moves in this puzzle should follow:
1. Only one disk can be moved at a time.
2. Each move consists of taking the upper disk from one stack and placing it on top of another stack.
3. A larger disk may not be placed on top of a smaller disk.
The goal is to move the entire stack to the third peg.
Example: With 3 disks numbered 1 (smallest), 2, and 3 (largest), the initial state is [[3, 2, 1], [], []], and a solution might be:
moves = [[1 , 0 , 2] , [2 , 0 , 1] , [1 , 2 , 1] , [3 , 0 , 2] , [1 , 1 , 0] , [2 , 1 , 2] , [1 , 0 , 2]]
This means: Move disk 1 from peg 0 to peg 2, then move disk 2 from peg 0 to peg 1, and so on. Requirements:
• When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
• The positions are 0-indexed (the leftmost peg is 0).
• Ensure your final answer includes the complete list of moves in the format: moves = [[disk id, from peg, to peg], ...]

       """
      },
      {"role": "user", "content":
       """
I have a puzzle with 7 disks of different sizes with Initial configuration:
• Peg 0: 7 (bottom), . . . 2, 1 (top)
• Peg 1: (empty)
• Peg 2: (empty)
Goal configuration:
• Peg 0: (empty)
• Peg 1: (empty)
• Peg 2: 7 (bottom), . . . 2, 1 (top)
Rules:
• Only one disk can be moved at a time.
• Only the top disk from any stack can be moved.
• A larger disk may not be placed on top of a smaller disk.
Find the sequence of moves to transform the initial configuration into the goal configuration.

       """},
    ]

completion = client.chat.completions.create(
  extra_headers={
  },
  extra_body={"enable_thinking": True},
  model="qwen/qwen3-32b",
  max_tokens=64000,
  messages=messages
)
print(completion.choices[0].message.content)

In [ ]:
from typing import List, Tuple

# ---------- 类型别名 ----------
Move = Tuple[int, int, int]   # (disk, src_peg, dst_peg)  ── 0·1·2 柱编号

# ---------- 校验器 ----------
def validate_hanoi_moves(moves: List[Move],
                         n_disks: int | None = None,     # ← 可以省略
                         use_one_based: bool = False,
                         verbose: bool = False) -> bool:

    if n_disks is None:
      try:
          n_disks = max(disk for disk, _, _ in moves)
      except ValueError:          # moves 为空
          raise ValueError("moves list is empty; cannot infer disk count")
    """
    检验动作序列能否把 N 盘汉诺塔从柱 0(或1) 完整合法地移动到柱 2(或3)。
    """
    # 选择下标映射
    if use_one_based:
        def idx(i: int) -> int:
            if i not in {1, 2, 3}:
                raise ValueError(f"Peg must be 1,2,3; got {i}")
            return i - 1
    else:
        def idx(i: int) -> int:
            if i not in {0, 1, 2}:
                raise ValueError(f"Peg must be 0,1,2; got {i}")
            return i

    # --- 构造初始 / 目标 ---
    pegs = [list(range(n_disks, 0, -1)), [], []]  # 初始：全部在柱 0
    goal = [[], [], list(range(n_disks, 0, -1))]  # 目标：全部在柱 2

    def top(p):                  # 取柱顶盘号；空柱返回 None
        return pegs[p][-1] if pegs[p] else None

    # --- 逐步检验 ---
    for step, (disk, src, dst) in enumerate(moves, start=1):
        s, d = idx(src), idx(dst)

        # 1) 盘号合法
        if not (1 <= disk <= n_disks):
            raise ValueError(f"Step {step}: disk {disk} out of range 1…{n_disks}")

        # 2) 源柱非空 & 顶端必须正是该盘
        if top(s) != disk:
            raise ValueError(
                f"Step {step}: top of peg {src} is {top(s)}, expected {disk}")

        # 3) 目标柱合法（小盘不能压在大盘下）
        if top(d) is not None and top(d) < disk:
            raise ValueError(
                f"Step {step}: cannot place disk {disk} on smaller disk {top(d)}")

        # 4) 执行移动
        pegs[s].pop()
        pegs[d].append(disk)

        if verbose:
            print(f"After step {step}: {pegs}")

    # --- 最终状态检查 ---
    if pegs != goal:
        raise AssertionError(
            f"Final state differs from goal.\nGot : {pegs}\nGoal: {goal}")

    if verbose:
        print("✓ All moves valid, goal reached!")
    return True


# ---------- 使用示例 ----------
if __name__ == "__main__":
    moves = [

    [1, 0, 2],
    [2, 0, 1],
    [1, 2, 1],
    [3, 0, 2],
    [1, 1, 0],
    [2, 1, 2],
    [1, 0, 2],
    [4, 0, 1],
    [1, 2, 1],
    [2, 0, 2],
    [1, 1, 0],
    [3, 2, 1],
    [1, 0, 2],
    [2, 2, 0],
    [1, 1, 0],
    [5, 0, 2],
    [1, 0, 1],
    [2, 2, 1],
    [1, 1, 2],
    [3, 0, 2],
    [1, 2, 1],
    [2, 1, 0],
    [1, 0, 2],
    [4, 1, 2],
    [1, 0, 1],
    [2, 2, 0],
    [1, 1, 2],
    [3, 1, 0],
    [1, 2, 1],
    [2, 0, 2],
    [1, 1, 0],
    [6, 0, 1],
    [1, 2, 0],
    [2, 1, 2],
    [1, 0, 2],
    [3, 1, 2],
    [1, 2, 0],
    [2, 2, 1],
    [1, 0, 1],
    [4, 0, 2],
    [1, 1, 0],
    [2, 2, 1],
    [1, 0, 1],
    [3, 2, 1],
    [1, 1, 2],
    [2, 1, 0],
    [1, 2, 0],
    [5, 0, 2],
    [1, 0, 1],
    [2, 2, 0],
    [1, 1, 2],
    [3, 0, 1],
    [1, 2, 0],
    [2, 1, 2],
    [1, 0, 2],
    [4, 1, 0],
    [1, 2, 1],
    [2, 0, 2],
    [1, 1, 2],
    [3, 1, 2],
    [1, 2, 0],
    [2, 2, 1],
    [1, 0, 1],
    [7, 0, 2],
    [1, 1, 2],
    [2, 0, 1],
    [1, 2, 1],
    [3, 0, 2],
    [1, 1, 0],
    [2, 1, 2],
    [1, 0, 2],
    [4, 1, 0],
    [1, 2, 1],
    [2, 2, 0],
    [1, 1, 0],
    [3, 2, 0],
    [1, 0, 2],
    [2, 1, 0],
    [1, 2, 0],
    [5, 1, 2],
    [1, 0, 1],
    [2, 2, 1],
    [1, 1, 2],
    [3, 0, 1],
    [1, 2, 0],
    [2, 1, 2],
    [1, 0, 2],
    [4, 0, 1],
    [1, 2, 0],
    [2, 2, 1],
    [1, 0, 1],
    [3, 2, 1],
    [1, 1, 0],
    [2, 1, 2],
    [1, 0, 2],
    [6, 1, 0],
    [1, 2, 1],
    [2, 0, 2],
    [1, 1, 2],
    [3, 0, 1],
    [1, 2, 0],
    [2, 2, 1],
    [1, 0, 1],
    [4, 2, 0],
    [1, 1, 0],
    [2, 1, 2],
    [1, 0, 2],
    [3, 1, 0],
    [1, 2, 1],
    [2, 2, 0],
    [1, 1, 0],
    [5, 2, 1],
    [1, 0, 2],
    [2, 1, 0],
    [1, 2, 0],
    [3, 1, 2],
    [1, 0, 1],
    [2, 0, 2],
    [1, 1, 2],
    [4, 0, 1],
    [1, 2, 0],
    [2, 2, 1],
    [1, 0, 1],
    [3, 2, 1],
    [1, 1, 2],
    [2, 1, 0],
    [1, 2, 0],
    [6, 0, 1],
    [1, 0, 2],
    [2, 1, 0],
    [1, 2, 1],
    [3, 0, 2],
    [1, 1, 0],
    [2, 2, 1],
    [1, 0, 1],
    [4, 0, 2],
    [1, 1, 0],
    [2, 1, 2],
    [1, 0, 2],
    [3, 1, 0],
    [1, 2, 1],
    [2, 2, 0],
    [1, 1, 0],
    [5, 0, 2],
    [1, 0, 1],
    [2, 2, 0],
    [1, 1, 2],
    [3, 0, 2],
    [1, 2, 0],
    [2, 0, 2],
    [1, 1, 2]
        ]

    validate_hanoi_moves(moves, verbose=True)  # ← 1-3 编号示例

    # 如果你的 moves 用 0-2 下标：
    # validate_hanoi_moves(moves_zero_based, n_disks=N, use_one_based=False, verbose=True)

After step 1: [[7, 6, 5, 4, 3, 2], [], [1]]
After step 2: [[7, 6, 5, 4, 3], [2], [1]]
After step 3: [[7, 6, 5, 4, 3], [2, 1], []]
After step 4: [[7, 6, 5, 4], [2, 1], [3]]
After step 5: [[7, 6, 5, 4, 1], [2], [3]]
After step 6: [[7, 6, 5, 4, 1], [], [3, 2]]
After step 7: [[7, 6, 5, 4], [], [3, 2, 1]]
After step 8: [[7, 6, 5], [4], [3, 2, 1]]
After step 9: [[7, 6, 5], [4, 1], [3, 2]]


ValueError: Step 10: top of peg 0 is 5, expected 2